In [17]:
import pandas as pd
import sqlalchemy as sql
import json
import numpy as np

In [3]:
from sql_connection import engine
engine.connect()

In [14]:
query_recptime = """
select 
    recpid,
    recpname,
    recpversion,
    trackintime,
    startrecptime,
    endrecptime,
    trackouttime, 
    case
        when endrecptime is not null and startrecptime is not null then trunc((endrecptime - startrecptime)*86400)
        when trackouttime is not null and trackintime is not null then trunc((trackouttime - trackintime)*86400)
        else 0
    end as recptime
from plld.hist@sgodsprd
where 
    prodarea in ('SINGAPORE','SINGAPORE3')
    and lottype in ('E', 'P')
    and TIMESTAMPTIME BETWEEN TO_TIMESTAMP('01-JAN-23 00.00.00', 'DD-MON-YY HH24.MI.SS') and TO_TIMESTAMP('31-DEC-24 23.59.59', 'DD-MON-YY HH24.MI.SS')
    and (
        (trackintime is not null and startrecptime is not null and endrecptime is not null and trackouttime is not null)
        or
        (trackintime is not null and trackouttime is not null and startrecptime is null and endrecptime is null)
    )
    and recpid is not null
"""

df_recptime = pd.read_sql(query_recptime, con=engine)
print(df_recptime.head())

     recpid recpname recpversion         trackintime       startrecptime  \
0  SWT02.01    SWT02         .01 2023-01-01 10:36:47 2023-01-01 10:36:47   
1  SOW04.01    SOW04         .01 2023-01-01 09:12:53 2023-01-01 09:12:53   
2  SOK03.06    SOK03         .06 2023-01-01 10:26:08 2023-01-01 10:26:08   
3  3BGC1.01    3BGC1         .01 2023-01-01 09:13:10 2023-01-01 09:13:10   
4  3CMA1.06    3CMA1         .06 2023-01-01 10:37:10 2023-01-01 10:37:10   

          endrecptime        trackouttime  recptime  
0 2023-01-01 10:36:49 2023-01-01 10:36:49         2  
1 2023-01-01 10:36:59 2023-01-01 10:36:59      5046  
2 2023-01-01 10:37:09 2023-01-01 10:37:09       661  
3 2023-01-01 10:37:10 2023-01-01 10:37:10      5039  
4 2023-01-01 10:37:10 2023-01-01 10:37:10         0  


In [ ]:
# drop null values
df_recptime = df_recptime[df_recptime['recpid'].notnull()]
# either all not null or trackin and trackout not null & startrecptime and endrecptime null
df_recptime = df_recptime[(df_recptime[['trackintime', 'startrecptime', 'endrecptime', 'trackouttime']].notnull().all(axis=1))
                           | (df_recptime[['trackintime', 'trackouttime']].notnull().all(axis=1) & df_recptime[['startrecptime', 'endrecptime']].isnull().all(axis=1))]

grouped = df_recptime.groupby('recpid')['recptime'].apply(list).reset_index()

# def calculate_statistics(times):
#     # remove top and tail 5% of times
#     sorted_times = np.sort(times)
#     n = len(sorted_times)
#     trimmed_times = sorted_times[int(0.05 * n): int(0.95 * n)]
#     count = len(trimmed_times)
#     # Calculation
#     mean = np.mean(trimmed_times)
#     std = np.std(trimmed_times)

#     lower_cutoff = np.min(trimmed_times)
#     upper_cutoff = np.max(trimmed_times)
#     # take less extreme of 2SD and min/max
#     lower_bound = max(mean - 2 * std, lower_cutoff)
#     upper_bound = min(mean + 2 * std, upper_cutoff)

#     return mean, std, count, lower_bound, upper_bound

# grouped['statistics'] = grouped['recptime'].apply(calculate_statistics)

# grouped[['mean', 'std', 'count', 'lower_bound', 'upper_bound']] = pd.DataFrame(
#     grouped['statistics'].tolist(), index=grouped.index
# )
# grouped.drop(columns=['statistics'], inplace=True)
# grouped.to_csv("RecipeTime.csv", index=False)
# print(grouped.head())


c:\Users\reny2\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\reny2\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\reny2\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\reny2\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\reny2\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity

In [7]:
query_recp = """
select 
    recpname,
    recpversion,
    activeflag
from plld.recp@sgodsprd
"""

df_recp = pd.read_sql(query_recp, con=engine)
print(df_recp.head())

  recpname recpversion activeflag
0    33RF1         .01           
1    36RF1         .01           
2    3AGC1         .01           
3    3AGC1         .02           
4    3AGC1         .03           


In [35]:
def get_active_recp(df_recp, recp_name):
    """Fetch the active row (ActiveFlag = 'A') from df_prcd based on prcd_name."""
    active_row = df_recp[(df_recp['recpname'] == recp_name) & (df_recp['activeflag'] == 'A')]
    return active_row.iloc[0]['recpversion'] if not active_row.empty else None

In [36]:
def fetch_recp_time(df_recp_time, recp_name, recp_version, SEED):
    """use the correct name and version to identify the range of recp time and generate a random recp time"""
    results = df_recp_time[(df_recp_time['recpname'] == recp_name) & (df_recp_time['recpversion'] == recp_version)]
    lower_bound = results.iloc[0]['lower_bound']
    upper_bound = results.iloc[0]['upper_bound']
    return np.random.uniform(lower_bound, upper_bound, SEED)